In [1]:
# %pip install peft
# %pip install evaluate

# 1. Import libraries and check GPU

In [2]:
import pandas as pd
import torch
from transformers import AutoTokenizer, RobertaConfig, RobertaForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
from peft import LoraConfig, get_peft_model
import numpy as np
import evaluate
from huggingface_hub import notebook_login

f:\APPLICATION\Python\Python312\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
f:\APPLICATION\Python\Python312\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
f:\APPLICATION\Python\Python312\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

# 2. Define evaluation metrics

In [4]:
accuracy = evaluate.load("accuracy")

In [5]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    # print(predictions, ":", labels)

    return accuracy.compute(predictions=predictions, references=labels)

In [6]:
df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vRJVBTys7CQjrbs6jcYDZEsEuMnMoDm0gGdZCX1tTvYxnbydPHNxfdooM4t3DeKGMMQ27HAvwpzFxKH/pub?output=csv").iloc[:, :2]
df

,comment,label
0,Đặt size S giao M.,NEU
1,Đèn cũ bụi bẩn như hàng đã qua sử dụng.,NEG
2,"Vải đẹp, màu sắc chất liệu y hình.",POS
3,.,NEU
4,Bên shopee đơt này bán và phí vận chuyển đắt h...,NEG
...,...,...
17995,"Màu vàng k giống ảnh, nhưng vải khá đẹp.",NEU
17996,Hàng y hình.,NEU
17997,sản phẩm y hình. chất cũng được.,POS
17998,Nice style but the fabric is too hard and ửink...,NEU


# 3. Preprocess dataset

In [7]:
def labels_index(string_labels):
  if string_labels == 'NEG':
    return 0
  elif string_labels == 'NEU':
    return 1
  else:
    return 2

In [8]:
checkpoint = "vinai/phobert-base-v2"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [9]:
def tokenize(data):
  return tokenizer(data, padding='longest', truncation=True)


print(tokenize("xin chào mọi người"))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': [0, 611, 3683, 207, 18, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1]}


In [10]:
tokenized_df = pd.DataFrame()
tokenized_df['comment'] = df['comment'].map(lambda r: list(tokenize(r).values()))
tokenized_df['label'] = df['label'].map(labels_index)
tokenized_df

,comment,label
0,"[[0, 8209, 19275, 2053, 574, 4015, 2], [0, 0, ...",1
1,"[[0, 15451, 633, 2694, 3116, 42, 119, 14, 89, ...",0
2,"[[0, 31385, 56234, 1487, 4, 412, 2104, 567, 19...",2
3,"[[0, 5, 2], [0, 0, 0], [1, 1, 1]]",1
4,"[[0, 559, 12231, 36653, 26202, 1204, 23, 170, ...",0
...,...,...
17995,"[[0, 6957, 354, 1947, 475, 15250, 2008, 4, 51,...",1
17996,"[[0, 1383, 1231, 50350, 18109, 2], [0, 0, 0, 0...",1
17997,"[[0, 4770, 3884, 1231, 50350, 18109, 567, 32, ...",2
17998,"[[0, 19518, 11303, 30045, 2292, 1866, 3377, 28...",1


In [11]:
length = len(tokenized_df.index)
train_dataset = tokenized_df.iloc[:int(length * 80 / 100),:]
valid_dataset = tokenized_df.iloc[int(length * 80 / 100):int(length * 90 / 100),:].reset_index()
test_dataset = tokenized_df.iloc[int(length * 90 / 100):,:].reset_index()

In [12]:
train_loader = []
valid_loader = []
test_loader = []

for i in range(len(train_dataset.index)):
  train_comment = torch.tensor([train_dataset['comment'][i]])
  train_label = torch.tensor([train_dataset['label'][i]])
  train_loader.append({'input_ids': train_comment[0][0], 'attention_mask': train_comment[0][1], 'labels': train_label[0]})

for i in range(len(valid_dataset.index)):
  valid_comment = torch.tensor([valid_dataset['comment'][i]])
  valid_label = torch.tensor([valid_dataset['label'][i]])
  valid_loader.append({'input_ids': valid_comment[0][0], 'attention_mask': valid_comment[0][1], 'labels': valid_label[0]})

for i in range(len(test_dataset.index)):
  test_comment = torch.tensor([test_dataset['comment'][i]])
  test_label = torch.tensor([test_dataset['label'][i]])
  test_loader.append({'input_ids': test_comment[0][0], 'attention_mask': test_comment[0][1], 'labels': test_label[0]})

# 4. Setup LoRA finetuning model

In [13]:
id2label = {
    0: "NEG",
    1: "NEU",
    2: "POS"
}
label2id = {
    "NEG": 0,
    "NEU": 1,
    "POS": 2
}

In [14]:
# checkpoint = "vinai/phobert-base-v2"
num_labels = 3
config = RobertaConfig.from_pretrained(checkpoint, num_labels=num_labels, id2label=id2label, label2id=label2id)
model = RobertaForSequenceClassification.from_pretrained(checkpoint, config=config).to(device)

print(model)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [15]:
# model = AutoModelForSequenceClassification.from_pretrained(
#     checkpoint, num_labels=3, id2label=id2label, label2id=label2id
# )

In [16]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later

In [17]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [18]:
target_modules = ['query','key','value','dense']
config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=target_modules,
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS"
)

model_lora = get_peft_model(model, config).to(device)
print_trainable_parameters(model_lora)

trainable params: 5950467 || all params: 141000198 || trainable%: 4.22018343548709


In [19]:
num_epoch = 8
batch_size = 16
learning_rate = 1e-5

In [20]:
training_args = TrainingArguments(
    evaluation_strategy="steps",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0,
    save_total_limit=10,
    num_train_epochs=num_epoch,
    fp16=False,
    push_to_hub=False,
    logging_steps=100,
    logging_first_step=True,
    logging_dir="logging/fa-sentiment-finetuned",
    eval_steps=100,
    resume_from_checkpoint = "checkpoint",
    output_dir = "checkpoint",
    load_best_model_at_end = True,
    save_steps = 100
)

In [21]:
trainer = Trainer(
    model=model_lora,
    train_dataset=train_loader,
    eval_dataset=valid_loader,
    args=training_args,
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer)
)
model_lora.config.use_cache = False  # silence the warnings. Please re-enable for inference!

f:\APPLICATION\Python\Python312\Lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


# 5. Train and evaluate

In [22]:
trainer.train()
trainer.save_model("best_finetuned_model")

  0%|          | 0/7200 [00:00<?, ?it/s]

{'loss': 1.1205, 'learning_rate': 9.998611111111112e-06, 'epoch': 0.0}
{'loss': 1.0987, 'learning_rate': 9.861111111111112e-06, 'epoch': 0.11}


  0%|          | 0/113 [00:00<?, ?it/s]

{'eval_loss': 1.0818744897842407, 'eval_accuracy': 0.38722222222222225, 'eval_runtime': 5.5066, 'eval_samples_per_second': 326.882, 'eval_steps_per_second': 20.521, 'epoch': 0.11}
{'loss': 1.0831, 'learning_rate': 9.722222222222223e-06, 'epoch': 0.22}


  0%|          | 0/113 [00:00<?, ?it/s]

...

{'eval_loss': 0.9232358932495117, 'eval_accuracy': 0.5677777777777778, 'eval_runtime': 3.4413, 'eval_samples_per_second': 523.059, 'eval_steps_per_second': 32.836, 'epoch': 7.78}
{'loss': 0.9248, 'learning_rate': 1.3888888888888888e-07, 'epoch': 7.89}


  0%|          | 0/113 [00:00<?, ?it/s]

{'eval_loss': 0.9228014349937439, 'eval_accuracy': 0.5683333333333334, 'eval_runtime': 3.5235, 'eval_samples_per_second': 510.859, 'eval_steps_per_second': 32.071, 'epoch': 7.89}
{'loss': 0.9261, 'learning_rate': 0.0, 'epoch': 8.0}


  0%|          | 0/113 [00:00<?, ?it/s]

{'eval_loss': 0.9228432774543762, 'eval_accuracy': 0.5683333333333334, 'eval_runtime': 3.3104, 'eval_samples_per_second': 543.747, 'eval_steps_per_second': 34.135, 'epoch': 8.0}
{'train_runtime': 930.067, 'train_samples_per_second': 123.862, 'train_steps_per_second': 7.741, 'train_loss': 0.9940477209621006, 'epoch': 8.0}


In [23]:
trainer.evaluate(eval_dataset=test_loader)

  0%|          | 0/113 [00:00<?, ?it/s]

{'eval_loss': 0.9417571425437927,
 'eval_accuracy': 0.5416666666666666,
 'eval_runtime': 3.4929,
 'eval_samples_per_second': 515.336,
 'eval_steps_per_second': 32.352,
 'epoch': 8.0}

In [24]:
# model_lora.push_to_hub("lora-vietnamese-feedback-analysis")

# 6. Inference

In [25]:
texts = [
    "Hàng rất tốt nha, cảm ơn shop.",
    "Áo tạm được, chất lượng vừa phải",
    "Hàng kém chất lượng, không nên mua",
    "Wow tuyệt quá!",
    "Phí tiền, khuyên các bạn nên tẩy chay shop.",
    "Quá xấu",
    "Màu k giống trong hình,Chất mặc nóng, bí không thấm hút mồ hôi.",
    "Chất da cá ( tiền naoc của đó ).",
    "Quần chưa thử nên không biết có che được không nhưng mình k thích cái viền ren lắm.",
    "Bình thường"
]
for text in texts:
  inputs = tokenizer.encode(text, return_tensors="pt").to("cuda")

  logits = model_lora(inputs).logits
  # print("---Logits:",logits)
  predictions = torch.max(logits,1).indices

  id2label = {0: "Negative", 1: "Neutral", 2: "Positive"}
  print(f"\"{text}\": \n>>> {id2label[predictions.tolist()[0]]}\n")

"Hàng rất tốt nha, cảm ơn shop.": 
>>> Positive

"Áo tạm được, chất lượng vừa phải": 
>>> Positive

"Hàng kém chất lượng, không nên mua": 
>>> Positive

"Wow tuyệt quá!": 
>>> Positive

"Phí tiền, khuyên các bạn nên tẩy chay shop.": 
>>> Positive

"Quá xấu": 
>>> Positive

"Màu k giống trong hình,Chất mặc nóng, bí không thấm hút mồ hôi.": 
>>> Positive

"Chất da cá ( tiền naoc của đó ).": 
>>> Positive

"Quần chưa thử nên không biết có che được không nhưng mình k thích cái viền ren lắm.": 
>>> Negative

"Bình thường": 
>>> Positive

